In [61]:
import requests
import folium
import pandas as pd
from sklearn.cluster import KMeans
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import json
import requests
from bs4 import BeautifulSoup as bSoup
import numpy as np
import re
from geopy.geocoders import Nominatim as geocoder
import csv
from pandas import json_normalize
import matplotlib.pyplot as mp
import seaborn as sns

print("Library import")
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
soup = bSoup(data,"html5lib")

contents = []
table = soup.find('table')
for row in table.findAll('td'):
    col = {}
    #Skip cells of table with not assinged.
    if row.span.text == 'Not assigned':
        pass
    #Split the contents of table cell, where first 3 chars are the postal code, and borough is contained outside the (),
    #and the neighbourhood is within the ().
    else:
        col['PCode'] = row.p.text[:3]
        col['Borough'] = (row.span.text).split('(')[0]
        col['Neighbourhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        contents.append(col)
tblCont = pd.DataFrame(contents)
tblCont['Borough'] = tblCont['Borough'].replace({'EtobicokeNorthwest':'Etobicoke Northwest',
                                            'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                            'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                            'MississaugaCanada Post Gateway Processing Centre':'Mississauga',
                                            'East YorkEast Toronto':'East York/East Toronto'})
print("Successful scrap")


Library import
Successful scrap


#### Csv file from the url contains the long and lat of Toronto, just have to parse out the ones that match the scraped postal codes.


In [2]:
longLatCSV = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
parsePCodes = pd.read_csv(longLatCSV)

#### Join the parsePCodes with tblCont on postal codes, and remove added columns from the dataframe that have NaN data, delete duplicate data, and reset the index.

In [50]:
finalizedLatLong = pd.merge(tblCont, parsePCodes, left_on = 'PCode', right_on = 'Postal Code')
finalizedLatLong = finalizedLatLong.drop('Postal Code', axis = 1)
finalizedLatLong = finalizedLatLong.drop_duplicates()
finalizedLatLong.index = range(len(finalizedLatLong))
finalizedLatLong.to_csv('finalizedLatLong.csv')
finalizedLatLong.head()

,PCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


#### Foursquare api info

In [104]:
LIMIT = 100
radius = 500
VERSION = 20220410
CLIENT_ID = 'LUQWIMZ52YZWINDV0F1C01R40VFQM2QGE3NV54U4X510FRXC'
CLIENT_SECRET = '35NN2UF2WG33WGN20XPJN20MWA44ZE3VFBNETVU55XCMEMEA'
couProvCity = 'Toronto, Ontario, Canada'

#### Use foursquare agent

In [5]:
gLoc = geocoder(user_agent = "AI-Proj-Key-2022")
cityLoc = gLoc.geocode(couProvCity)
latitude = cityLoc.latitude
longitude = cityLoc.longitude
tlongitude = longitude
tlatitude = latitude

mapOfRegion = folium.Map(location = [latitude, longitude], zoom_start = 11)
#mapOfRegion

#### Place markers on neighbourhoods that have been scraped.

In [55]:
for lati, longi, borough, neighbourhood in zip(finalizedLatLong['Latitude'], finalizedLatLong['Longitude'],finalizedLatLong['Borough'],finalizedLatLong['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lati, longi],
    fill = True,
    fill_opacity = 0.1,
    color = '#cc8131',
    fill_color = 'yellow',
    radius = 6,
    popup = label,
    parse_html = False).add_to(mapOfRegion)
mapOfRegion

#### Reading the needed csv files to datasets to join on common columns.

In [52]:
dataCrime = pd.read_csv ('Neighbourhood_Crime_Rates_(Boundary_File)_.csv')
dataNeighProfiles = pd.read_csv ('neighbourhood-profiles-2016-csv.csv')
dataNeigh = pd.read_csv ('Neighbourhoods-4326.csv')
#print(dataNeighProfiles)

#### Joining dataNeigh with dataCrime on the same Neighbourhood name and AREA_DESC name.

In [54]:
crimeAndNeigh = dataCrime.merge(dataNeigh,left_on = 'Neighbourhood', right_on = 'AREA_DESC')
crimeAndNeigh = pd.merge(crimeAndNeigh, finalizedLatLong, on = 'Neighbourhood')
crimeAndNeigh.head()
#print(crimeAndNeigh)

,OBJECTID_x,Neighbourhood,Hood_ID,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,Assault_AVG,Assault_CHG,Assault_Rate_2019,AutoTheft_2014,AutoTheft_2015,AutoTheft_2016,AutoTheft_2017,AutoTheft_2018,AutoTheft_2019,AutoTheft_AVG,AutoTheft_CHG,AutoTheft_Rate_2019,BreakandEnter_2014,BreakandEnter_2015,BreakandEnter_2016,BreakandEnter_2017,BreakandEnter_2018,BreakandEnter_2019,BreakandEnter_AVG,BreakandEnter_CHG,BreakandEnter_Rate_2019,Homicide_2014,Homicide_2015,Homicide_2016,Homicide_2017,Homicide_2018,Homicide_2019,Homicide_AVG,Homicide_CHG,Homicide_Rate_2019,Robbery_2014,Robbery_2015,Robbery_2016,Robbery_2017,Robbery_2018,Robbery_2019,Robbery_AVG,Robbery_CHG,Robbery_Rate_2019,TheftOver_2014,TheftOver_2015,TheftOver_2016,TheftOver_2017,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area_x,Shape__Length_x,AREA_ATTR_ID,AREA_DESC,AREA_ID,AREA_LONG_CODE,AREA_NAME,AREA_SHORT_CODE,CLASSIFICATION,CLASSIFICATION_CODE,LATITUDE,LONGITUDE,OBJECTID_y,PARENT_AREA_ID,Shape__Area_y,Shape__Length_y,X,Y,_id,geometry,PCode,Borough,Latitude,Longitude
0,7,The Beaches,63,21567,83,108,86,111,94,81,93.8,-0.14,375.6,8,23,7,10,30,20,16.3,-0.33,92.7,44,42,77,33,43,57,49.3,0.33,264.3,0,0,0,0,0,0,0.0,0.0,0.0,15,14,19,48,10,16,20.3,0.6,74.2,8,5,3,7,5,9,6.2,0.80,41.7,3.595829e+06,11275.180743,26005453,The Beaches,2480073,63,The Beaches (63),63,NaN,NaN,NaN,NaN,17547105,NaN,6.877394e+06,15586.297670,NaN,NaN,126,"{'type': 'Polygon', 'coordinates': (((-79.3148...",M4E,East Toronto,43.676357,-79.293031
1,9,Thorncliffe Park,55,21108,86,105,87,99,93,115,97.5,0.24,544.8,12,11,4,7,9,13,9.3,0.44,61.6,33,27,24,19,23,27,25.5,0.17,127.9,3,1,1,4,0,0,1.5,0.0,0.0,16,7,16,9,5,13,11.0,1.6,61.6,9,6,11,9,3,5,7.2,0.67,23.7,3.126554e+06,7502.709430,26005455,Thorncliffe Park,2480075,55,Thorncliffe Park (55),55,Neighbourhood Improvement Area,NIA,NaN,NaN,17547073,NaN,5.987291e+06,10380.556660,NaN,NaN,124,"{'type': 'Polygon', 'coordinates': (((-79.3354...",M4H,East York,43.705369,-79.349372
2,11,Humewood-Cedarvale,106,14365,43,52,52,38,47,46,46.3,-0.02,320.2,11,7,17,22,24,16,16.2,-0.33,111.4,31,39,27,39,33,26,32.5,-0.21,181.0,0,1,0,1,0,0,0.3,0.0,0.0,8,11,8,6,4,10,7.8,1.5,69.6,3,3,5,6,7,4,4.7,-0.43,27.8,1.871263e+06,6036.268116,26005559,Humewood-Cedarvale,2480179,106,Humewood-Cedarvale (106),106,NaN,NaN,NaN,NaN,17545985,NaN,3.581523e+06,8355.083706,NaN,NaN,56,"{'type': 'Polygon', 'coordinates': (((-79.4184...",M6C,York,43.693781,-79.428191
3,15,Scarborough Village,139,16724,161,153,192,152,178,175,168.5,-0.02,1046.4,20,15,9,21,20,18,17.2,-0.10,107.6,49,48,34,32,45,40,41.3,-0.11,239.2,1,0,6,1,0,0,1.3,0.0,0.0,30,33,22,23,14,28,25.0,1.0,167.4,3,2,2,4,3,2,2.7,-0.33,12.0,3.151015e+06,7922.764442,26005505,Scarborough Village,2480125,139,Scarborough Village (139),139,Neighbourhood Improvement Area,NIA,NaN,NaN,17545585,NaN,6.040096e+06,10973.560770,NaN,NaN,31,"{'type': 'Polygon', 'coordinates': (((-79.2139...",M1J,Scarborough,43.744734,-79.239476
4,20,Humber Summit,21,12416,76,89,118,116,109,118,104.3,0.08,950.4,66,42,62,83,106,135,82.3,0.27,1087.3,54,44,41,40,56,57,48.7,0.02,459.1,0,1,0,2,1,3,1.2,2.0,24.2,22,23,23,27,31,28,25.7,-0.1,225.5,15,16,18,18,15,22,17.3,0.47,177.2,7.966905e+06,12608.573118,26005514,Humber Summit,2480134,21,Humber Summit (21),21,Neighbourhood Improvement Area,NIA,NaN,NaN,17545329,NaN,1.528418e+07,17453.251550,NaN,NaN,15,"{'type': 'Polygon', 'coordinates': (((-79.5806...",M9L,North York,43.756303,-79.565963


#### Merging crimeAndNeigh, and datatNeighProfiles together.

In [148]:
totalDF = pd.merge(crimeAndNeigh, dataNeighProfiles, on = 'Neighbourhood')
totalDF.to_csv('totalDF.csv')

#### Print out columns to find correlation criteria.

In [57]:
features1 = list(["BreakandEnter_2015","Divorced","$30,000 to $39,999"])
print(totalDF.columns.tolist())
#totalDF[features1].corr()

['OBJECTID_x', 'Neighbourhood', 'Hood_ID', 'Population', 'Assault_2014', 'Assault_2015', 'Assault_2016', 'Assault_2017', 'Assault_2018', 'Assault_2019', 'Assault_AVG', 'Assault_CHG', 'Assault_Rate_2019', 'AutoTheft_2014', 'AutoTheft_2015', 'AutoTheft_2016', 'AutoTheft_2017', 'AutoTheft_2018', 'AutoTheft_2019', 'AutoTheft_AVG', 'AutoTheft_CHG', 'AutoTheft_Rate_2019', 'BreakandEnter_2014', 'BreakandEnter_2015', 'BreakandEnter_2016', 'BreakandEnter_2017', 'BreakandEnter_2018', 'BreakandEnter_2019', 'BreakandEnter_AVG', 'BreakandEnter_CHG', 'BreakandEnter_Rate_2019', 'Homicide_2014', 'Homicide_2015', 'Homicide_2016', 'Homicide_2017', 'Homicide_2018', 'Homicide_2019', 'Homicide_AVG', 'Homicide_CHG', 'Homicide_Rate_2019', 'Robbery_2014', 'Robbery_2015', 'Robbery_2016', 'Robbery_2017', 'Robbery_2018', 'Robbery_2019', 'Robbery_AVG', 'Robbery_CHG', 'Robbery_Rate_2019', 'TheftOver_2014', 'TheftOver_2015', 'TheftOver_2016', 'TheftOver_2017', 'TheftOver_2018', 'TheftOver_2019', 'TheftOver_AVG', 'T

#### Getting correlation between B & E 2015, income of 30k - 39k, and Male 20-24 yrs old

In [11]:
totalDF["    $30,000 to $39,999"] = totalDF["    $30,000 to $39,999"].str.replace(',','')
totalDF["Male: 20 to 24 years"] = pd.to_numeric(totalDF["Male: 20 to 24 years"])
totalDF["    $30,000 to $39,999"] = pd.to_numeric(totalDF["    $30,000 to $39,999"])
correlation1 = totalDF[["BreakandEnter_2015", "    $30,000 to $39,999", "Male: 20 to 24 years"]].corr()
correlation1


,BreakandEnter_2015,"$30,000 to $39,999",Male: 20 to 24 years
BreakandEnter_2015,1.000000,0.645467,0.779515
"$30,000 to $39,999",0.645467,1.000000,0.857844
Male: 20 to 24 years,0.779515,0.857844,1.000000


In [12]:
totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"] = totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"].str.replace(',','')
totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"] = pd.to_numeric(totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"])
totalDF["  18 to 64 years (%).1"] = pd.to_numeric(totalDF["  18 to 64 years (%).1"])
totalDF["  Postsecondary certificate, diploma or degree"] = pd.to_numeric(totalDF["  Postsecondary certificate, diploma or degree"])
#Below is removes commas from the column.
#totalDF[" Jamaican"] = totalDF[" Jamaican"].str.replace(',','')
correlation2 = totalDF[["  Postsecondary certificate, diploma or degree", "Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data", "  18 to 64 years (%).1"]].corr()
correlation2

,"Postsecondary certificate, diploma or degree",Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data,18 to 64 years (%).1
"Postsecondary certificate, diploma or degree",1.000000,0.879337,0.112466
Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data,0.879337,1.000000,0.159046
18 to 64 years (%).1,0.112466,0.159046,1.000000


In [49]:
#totalDF["    Migrants"] = totalDF["    Migrants"].str.replace(',','')
#totalDF["  English only"] = totalDF["  English only"].str.replace(',','')
totalDF["            Italian"] = totalDF["            Italian"].str.replace(',','')
#totalDF["      Social assistance benefits: Population with an amount"] = totalDF["      Social assistance benefits: Population with an amount"].str.replace(',','')
#totalDF["Population density per square kilometre"] = totalDF["Population density per square kilometre"].str.replace(',','')
#totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"] = totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"].str.replace(',','')
totalDF["            Italian"] = pd.to_numeric(totalDF["            Italian"])
totalDF["    Divorced"] = pd.to_numeric(totalDF["    Divorced"])
totalDF["    Migrants"] = pd.to_numeric(totalDF["    Migrants"])
totalDF["Assault_2015"] = pd.to_numeric(totalDF["Assault_2015"])
totalDF["  English only"] = pd.to_numeric(totalDF["  English only"])
totalDF["  No certificate, diploma or degree"] = pd.to_numeric(totalDF["  No certificate, diploma or degree"])
totalDF["  Secondary (high) school diploma or equivalency certificate.1"] = pd.to_numeric(totalDF["  Secondary (high) school diploma or equivalency certificate.1"])
totalDF["    University certificate, diploma or degree at bachelor level or above.1"] = pd.to_numeric(totalDF["    University certificate, diploma or degree at bachelor level or above.1"])
totalDF["      Social assistance benefits: Population with an amount"] = pd.to_numeric(totalDF["      Social assistance benefits: Population with an amount"])
totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"] = pd.to_numeric(totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"])
totalDF["Population density per square kilometre"] = pd.to_numeric(totalDF["Population density per square kilometre"])
totalDF["Male: 30 to 34 years"] = pd.to_numeric(totalDF["Male: 30 to 34 years"])
totalDF["Female: 30 to 34 years"] = pd.to_numeric(totalDF["Female: 20 to 24 years"])
correlation3 = totalDF[["Male: 30 to 34 years", "Female: 30 to 34 years","Assault_2015","    Migrants", "Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data", "Population density per square kilometre", "      Social assistance benefits: Population with an amount", "    Divorced", "    University certificate, diploma or degree at bachelor level or above.1", "  Secondary (high) school diploma or equivalency certificate.1", "  No certificate, diploma or degree", "  English only", "            Italian"]].corr()
correlation3


,Male: 30 to 34 years,Female: 30 to 34 years,Assault_2015,Migrants,Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data,Population density per square kilometre,Social assistance benefits: Population with an amount,Divorced,"University certificate, diploma or degree at bachelor level or above.1",Secondary (high) school diploma or equivalency certificate.1,"No certificate, diploma or degree",English only,Italian
Male: 30 to 34 years,1.000000,0.820427,0.733709,0.918826,0.774026,0.384195,0.330541,0.737052,0.907673,0.514421,0.218445,0.747344,0.022526
Female: 30 to 34 years,0.820427,1.000000,0.836241,0.875081,0.862433,0.243872,0.456311,0.762187,0.758169,0.665732,0.429394,0.855054,0.094737
Assault_2015,0.733709,0.836241,1.000000,0.710655,0.646270,0.296756,0.580030,0.637049,0.573660,0.546582,0.366503,0.648223,0.149874
Migrants,0.918826,0.875081,0.710655,1.000000,0.741815,0.355906,0.221499,0.676311,0.920981,0.409422,0.108243,0.720314,-0.039405
Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data,0.774026,0.862433,0.646270,0.741815,1.000000,0.070440,0.586781,0.917165,0.721188,0.866430,0.636187,0.994082,0.144299
Population density per square kilometre,0.384195,0.243872,0.296756,0.355906,0.070440,1.000000,0.137191,0.178639,0.349684,-0.080430,-0.172781,0.036900,-0.209982
Social assistance benefits: Population with an amount,0.330541,0.456311,0.580030,0.221499,0.586781,0.137191,1.000000,0.656002,0.084288,0.778936,0.767973,0.613213,0.215641
Divorced,0.737052,0.762187,0.637049,0.676311,0.917165,0.178639,0.656002,1.000000,0.665349,0.803244,0.601248,0.905919,0.176000
"University certificate, diploma or degree at bachelor level or above.1",0.907673,0.758169,0.573660,0.920981,0.721188,0.349684,0.084288,0.665349,1.000000,0.315919,-0.019803,0.687046,-0.127198
Secondary (high) school diploma or equivalency certificate.1,0.514421,0.665732,0.546582,0.409422,0.866430,-0.080430,0.778936,0.803244,0.315919,1.000000,0.887087,0.878074,0.279940


#### Getting foursquare venue information

In [162]:
LIMIT = 100
radius = 500
VERSION = 20220410
CLIENT_ID = 'LUQWIMZ52YZWINDV0F1C01R40VFQM2QGE3NV54U4X510FRXC'
CLIENT_SECRET = '35NN2UF2WG33WGN20XPJN20MWA44ZE3VFBNETVU55XCMEMEA'



def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]#['groups'][0]['items']
    
    
    
        
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    return (venues_list)



In [167]:
LIMIT = 50
radius = 500
VERSION = 20220410
CLIENT_ID = 'LUQWIMZ52YZWINDV0F1C01R40VFQM2QGE3NV54U4X510FRXC'
CLIENT_SECRET = '35NN2UF2WG33WGN20XPJN20MWA44ZE3VFBNETVU55XCMEMEA'
category = 12094
url = "https://api.foursquare.com/v3/places/search?&ll={}{}&radius={}&categories{}&limit={}"

headers = {"Accept": "application/json", "Authorization": "fsq3f6rpL1EABnqQ6Ie0wxtij4eZs5OX+vMZtdncijsSbNM="}
response = requests.request("GET", url, headers = headers)
res = response.json()
res


{'message': 'Invalid lat long found, format expected: `12.34,56.78`, got: `{}{}`'}

In [163]:
antwerp_venues = getNearbyVenues(names = totalDF['Neighbourhood'],
                                 latitudes = totalDF['Latitude'],
                                 longitudes = totalDF['Longitude'])


The Beaches
Thorncliffe Park
Humewood-Cedarvale
Scarborough Village
Humber Summit
Victoria Village
Hillcrest Village
Willowdale West
Woburn
Bayview Village
Weston


In [165]:
antwerp_venues

[[], [], [], [], [], [], [], [], [], [], []]